# Running the pipeline in a notebook

In [114]:
# imports
import os
import sys
import pprint

# project path
target_dir = os.path.dirname(os.getcwd())
target_dir_fraud = target_dir + "/fraud"
sys.path.extend([target_dir, target_dir_fraud])

import tfx
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.proto import example_gen_pb2
from tfx.v1 import proto
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.wrappers_pb2 import BoolValue

from configs import PROJECT_DIR, OUTPUT_DIR, TENSORBOARD_LOG_DIR, TRAIN_NUM_STEPS, EVAL_NUM_STEPS, SLICE_BY, LABEL_KEY, EVAL_ACCURACY_THRESHOLD
from local_runner import DATA_PATH, SERVING_MODEL_DIR

pp = pprint.PrettyPrinter()

# context
context = InteractiveContext(pipeline_root=OUTPUT_DIR)

## Load data

In [3]:
# load the data
output_config = proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        proto.SplitConfig.Split(name='train', hash_buckets=3),
        proto.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))
example_gen = tfx.components.CsvExampleGen(input_base=DATA_PATH, output_config=output_config)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 17
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 27
        type_id: 14
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/CsvExampleGen/examples/17"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:505381,xor_checksum:1631890618,sum_checksum:1631890618"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [4]:
# let us examine the outputs:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/CsvExampleGen/examples/17


In [5]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "AMT_CREDIT"
    value {
      float_list {
        value: 180000.0
      }
    }
  }
  feature {
    key: "AMT_INCOME_TOTAL"
    value {
      float_list {
        value: 72000.0
      }
    }
  }
  feature {
    key: "CNT_CHILDREN"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "CODE_GENDER"
    value {
      bytes_list {
        value: "F"
      }
    }
  }
  feature {
    key: "FLAG_MOBIL"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "NAME_CONTRACT_TYPE"
    value {
      bytes_list {
        value: "Cash loans"
      }
    }
  }
  feature {
    key: "NAME_TYPE_SUITE"
    value {
      bytes_list {
        value: "Unaccompanied"
      }
    }
  }
  feature {
    key: "TARGET"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "AMT_CREDIT"
    value {
      float_list {
        value: 45000.0
      }
    }
  }
  feature {

2021-09-25 16:13:32.823764: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-25 16:13:32.859586: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


## Generate statistics

In [6]:
# statistics
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'], exclude_splits=None)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 18
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 28
        type_id: 16
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/StatisticsGen/statistics/18"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
context.show(statistics_gen.outputs['statistics'])

## Generate schema

In [8]:
# schema
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 19
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 29
        type_id: 18
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/SchemaGen/schema/19"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [9]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'CODE_GENDER',STRING,required,,'CODE_GENDER'
'NAME_CONTRACT_TYPE',STRING,required,,'NAME_CONTRACT_TYPE'
'NAME_TYPE_SUITE',STRING,required,,'NAME_TYPE_SUITE'
'AMT_CREDIT',FLOAT,required,,-
'AMT_INCOME_TOTAL',FLOAT,required,,-
'CNT_CHILDREN',INT,required,,-
'FLAG_MOBIL',INT,required,,-
'TARGET',INT,required,,-


/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'CODE_GENDER',"'F', 'M'"
'NAME_CONTRACT_TYPE',"'Cash loans', 'Revolving loans'"
'NAME_TYPE_SUITE',"'Children', 'Family', 'Group of people', 'Other_A', 'Other_B', 'Spouse, partner', 'Unaccompanied'"


## Validate data

In [10]:
# example validation
example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 20
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 30
        type_id: 20
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/ExampleValidator/anomalies/20"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
context.show(example_validator.outputs['anomalies'])

/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Transform data

In [12]:
# transform using preprocessing_fn
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=PROJECT_DIR + '/fraud/preprocessing.py'
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying configs.py -> build/lib
copying features.py -> build/lib
copying local_runner.py -> build/lib
copying preprocessing.py -> build/lib
copying pipeline.py -> build/lib
installing to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
running install
running install_lib
copying build/lib/configs.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
copying build/lib/features.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
copying build/lib/local_runner.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
copying build/lib/preprocessing.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
copying build/lib/pipeline.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp8l5tpaye
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_

Instructions for updating:
Use ref() instead.


Processing /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/_wheels/tfx_user_code_Transform-0.0+fe57464482e94319cf65ecae85f66bfd487675e50a396930a7a17bbadbcd0b05-py3-none-any.whl


2021-09-25 16:14:59.905926: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/21/.temp_path/tftransform_tmp/073db575ae0a45a589dc82d6ba4ce538/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/21/.temp_path/tftransform_tmp/073db575ae0a45a589dc82d6ba4ce538/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/21/.temp_path/tftransform_tmp/a9fff8f373d0412a9d60a947612464f2/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/21/.temp_path/tftransform_tmp/a9fff8f373d0412a9d60a947612464f2/assets


ExecutionResult(
    component_id: Transform
    execution_id: 21
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 31
        type_id: 22
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/21"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 32
        type_id: 14
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transformed_examples/21"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 33
        type_id: 23
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/updated_analyzer_cache/21"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 34
        type_id: 18
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/pre_transform_schema/21"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
       

In [13]:
# training
trainer_args = {
    'module_file': PROJECT_DIR + '/fraud/estimators/wide_and_deep.py',
    'examples': transform.outputs['transformed_examples'],
    'schema': schema_gen.outputs['schema'],
    'transform_graph': transform.outputs['transform_graph'],
    'train_args': tfx.v1.proto.TrainArgs(num_steps=TRAIN_NUM_STEPS),
    'eval_args': tfx.v1.proto.EvalArgs(num_steps=EVAL_NUM_STEPS),
}
trainer = tfx.components.Trainer(**trainer_args)
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying wide_and_deep.py -> build/lib
installing to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmptuj9ihpk
running install
running install_lib
copying build/lib/wide_and_deep.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmptuj9ihpk
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmptuj9ihpk/tfx_user_code_Trainer-0.0+2f58982baeeffd29ca2bd7ce376ffbbe0dcb9c7b3aa6a6a48c0cc2b

Processing /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/_wheels/tfx_user_code_Trainer-0.0+2f58982baeeffd29ca2bd7ce376ffbbe0dcb9c7b3aa6a6a48c0cc2b496c77d1e-py3-none-any.whl


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50


...

Epoch 14/50
100/100 [==============================] - ETA: 0s - loss: 0.3566 - binary_accuracy: 0.8880 - auc: 0.499 - ETA: 0s - loss: 0.2757 - binary_accuracy: 0.9210 - auc: 0.544 - ETA: 0s - loss: 0.2803 - binary_accuracy: 0.9193 - auc: 0.534 - ETA: 0s - loss: 0.2803 - binary_accuracy: 0.9192 - auc: 0.540 - ETA: 0s - loss: 0.2810 - binary_accuracy: 0.9189 - auc: 0.541 - 0s 5ms/step - loss: 0.2818 - binary_accuracy: 0.9186 - auc: 0.5426 - val_loss: 0.2759 - val_binary_accuracy: 0.9215 - val_auc: 0.5186
Epoch 15/50
100/100 [==============================] - ETA: 0s - loss: 0.2869 - binary_accuracy: 0.9160 - auc: 0.538 - ETA: 0s - loss: 0.2773 - binary_accuracy: 0.9205 - auc: 0.538 - ETA: 0s - loss: 0.2855 - binary_accuracy: 0.9170 - auc: 0.540 - ETA: 0s - loss: 0.2825 - binary_accuracy: 0.9183 - auc: 0.539 - ETA: 0s - loss: 0.2839 - binary_accuracy: 0.9176 - auc: 0.543 - 0s 5ms/step - loss: 0.2827 - binary_accuracy: 0.9181 - auc: 0.5431 - val_loss: 0.2759 - val_binary_accuracy: 0.9215 

100/100 [==============================] - ETA: 0s - loss: 0.2555 - binary_accuracy: 0.9280 - auc: 0.620 - ETA: 0s - loss: 0.2860 - binary_accuracy: 0.9164 - auc: 0.551 - ETA: 0s - loss: 0.2764 - binary_accuracy: 0.9205 - auc: 0.545 - ETA: 0s - loss: 0.2780 - binary_accuracy: 0.9197 - auc: 0.553 - ETA: 0s - loss: 0.2812 - binary_accuracy: 0.9185 - auc: 0.548 - ETA: 0s - loss: 0.2818 - binary_accuracy: 0.9182 - auc: 0.552 - ETA: 0s - loss: 0.2817 - binary_accuracy: 0.9183 - auc: 0.551 - 1s 6ms/step - loss: 0.2815 - binary_accuracy: 0.9183 - auc: 0.5507 - val_loss: 0.2758 - val_binary_accuracy: 0.9214 - val_auc: 0.5165
Epoch 29/50
100/100 [==============================] - ETA: 0s - loss: 0.3113 - binary_accuracy: 0.9080 - auc: 0.460 - ETA: 0s - loss: 0.2837 - binary_accuracy: 0.9173 - auc: 0.551 - ETA: 0s - loss: 0.2809 - binary_accuracy: 0.9186 - auc: 0.548 - ETA: 0s - loss: 0.2804 - binary_accuracy: 0.9188 - auc: 0.551 - ETA: 0s - loss: 0.2790 - binary_accuracy: 0.9193 - auc: 0.554 - 

Epoch 42/50
100/100 [==============================] - ETA: 0s - loss: 0.2271 - binary_accuracy: 0.9400 - auc: 0.589 - ETA: 0s - loss: 0.2878 - binary_accuracy: 0.9155 - auc: 0.557 - ETA: 0s - loss: 0.2840 - binary_accuracy: 0.9171 - auc: 0.557 - ETA: 0s - loss: 0.2813 - binary_accuracy: 0.9184 - auc: 0.553 - ETA: 0s - loss: 0.2825 - binary_accuracy: 0.9178 - auc: 0.555 - 0s 5ms/step - loss: 0.2842 - binary_accuracy: 0.9171 - auc: 0.5536 - val_loss: 0.2762 - val_binary_accuracy: 0.9216 - val_auc: 0.5079
Epoch 43/50
100/100 [==============================] - ETA: 0s - loss: 0.2514 - binary_accuracy: 0.9320 - auc: 0.503 - ETA: 0s - loss: 0.2775 - binary_accuracy: 0.9196 - auc: 0.568 - ETA: 0s - loss: 0.2789 - binary_accuracy: 0.9192 - auc: 0.559 - ETA: 0s - loss: 0.2832 - binary_accuracy: 0.9175 - auc: 0.550 - ETA: 0s - loss: 0.2819 - binary_accuracy: 0.9181 - auc: 0.552 - ETA: 0s - loss: 0.2802 - binary_accuracy: 0.9187 - auc: 0.556 - 1s 6ms/step - loss: 0.2809 - binary_accuracy: 0.9184

INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/22/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 22
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 39
        type_id: 25
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 40
        type_id: 26
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model_run/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [14]:
%reload_ext tensorboard
%tensorboard --logdir {TENSORBOARD_LOG_DIR}

## Evaluation

In [30]:
# first lets add a model resolver
model_resolver = tfx.v1.dsl.Resolver(
    strategy_class=tfx.v1.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
).with_id('latest_blessed_model_resolver')
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 23
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [42]:
eval_config = tfma.EvalConfig(
    options=tfma.Options(include_default_metrics=BoolValue(value=True)),
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=f'{LABEL_KEY}_xf',
            preprocessing_function_names=['transform_features'])
    ],
    slicing_specs=[tfma.SlicingSpec()] + [tfma.SlicingSpec(feature_keys=[spec]) for spec in SLICE_BY],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='AUC',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': EVAL_ACCURACY_THRESHOLD}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10})))
        ])
    ])
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)
context.run(evaluator)

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1853be340> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x1853ca8e0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1853be340> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x1853ca8e0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x185ae29d0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x185b052b0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x185ae29d0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x185b052b0>).


...


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x18770e520> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x1876d41f0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x18770e520> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x1876d41f0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187c662e0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x187c495e0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187c662e0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x187c495e0>).


ExecutionResult(
    component_id: Evaluator
    execution_id: 27
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 47
        type_id: 29
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Evaluator/evaluation/27"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 48
        type_id: 30
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Evaluator/blessing/27"
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/22"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 39
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [43]:
context.show(evaluator.outputs['evaluation'])

In [47]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result, slicing_column=SLICE_BY[0])

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'CNT_CHILDREN:0', 'me…

In [109]:
blessing_loc = evaluator.outputs['blessing'].get()[0].uri
! ls {blessing_loc}

NOT_BLESSED


## Pushing the model

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [115]:
pusher_args = {'model': trainer.outputs['model'], 'model_blessing': evaluator.outputs['blessing'],
               'push_destination': tfx.v1.proto.PushDestination(
                   filesystem=tfx.v1.proto.PushDestination.Filesystem(base_directory=SERVING_MODEL_DIR))}
pusher = tfx.components.Pusher(**pusher_args)
context.run(pusher)

I0925 17:54:34.140743 288665088 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: Pusher
    execution_id: 28
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 49
        type_id: 32
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Pusher/pushed_model/28"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [124]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
if os.listdir(push_uri):
    latest_version = max(os.listdir(push_uri))
    latest_version_path = os.path.join(push_uri, latest_version)
    model = tf.saved_model.load(latest_version_path)
    for item in model.signatures.items():
        pp.pprint(item)